<a href="https://colab.research.google.com/github/KuJoJoTaRo2019/HaveFun/blob/master/Project1_short_semester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project1 is a web crawler.**

It contains:
1. Chinese University Rankings
2. Price Compare of TaoBao Commodity
3. Further method to crawl News Web
---

##**Chinese University Rankings**

In [ ]:
import requests
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
url = 'http://www.zuihaodaxue.cn/zuihaodaxuepaiming2020.html'

# get response from the web
def getHtmlText(url):
  try:
    r = requests.get(url, timeout = 30)
    r.raise_for_status()
    r.encoding = r.apparent_encoding # decide encoding way by the contents
    soup = BeautifulSoup(r.text, 'html.parser') # use BeautifulSoup to create a new object
    return soup
  except:
    return 'HaHaHa, an Error occurs!' # report when failing to get

# get data that we want
def pick_data(soup):
  temp = np.array(['rank','name','score'])
  for child in soup.tr.find_all_next('tr'): # find every child of 'tr'
    s1 = child.contents[1].string # rank
    s2 = child.contents[3].string # name
    s3 = child.contents[9].string # score
    tem = np.array([s1,s2,s3])
    temp = np.vstack((temp,tem)) # append one to the array
  return temp

In [ ]:
# Main
soup = getHtmlText(url)
result = pick_data(soup)
np.savetxt('Chinese_University_Rankings.csv',result,fmt='%s',delimiter=',')

##**Price Compare of TaoBao Commodity**

In [ ]:
import requests
import re
import numpy as np

In [ ]:
url = 'https://list.tmall.com/search_product.htm'

keyword='switch' # search 'switch' on TianMao
def getHtmlText_re(url):
  try:
      kv={'q':keyword}
      k={'User-Agent':'baiduspider'}   
      r=requests.get(url,params=kv,headers=k)
      # r=requests.get(url,params=kv,headers=k)
      r.raise_for_status()
      # r.encoding = r.apparent_encoding
      r.encoding = 'gbk' # use apparent_encoding will lead to a print error
      # print(r.request.headers)
      # print(r.request.url)
      print('The First time to request succeed.')
  except:
      print('HaHaHa, an Error occurs!')
  return r

In [ ]:
# To delete <...> in store name
def rename(get_storename):
  flag = True
  it = 0
  for i in get_storename:
    ch = ''
    if '<' in i:
      for j in i:
        if j == '<':
          flag = False
        if flag:
          ch += j
        if j == '>':
          flag = True
      get_storename[it] = ch
    it += 1 
  return get_storename

In [ ]:
# use re to get lots of information in one page
def get_one_page(r_new):
  get_price = re.findall(r'<em title="([0-9]+?.[0-9]+?)">',r_new.text)
  get_itemname = re.findall(r'target="_blank" title="(.+?)"',r_new.text)
  get_itemname = get_itemname[1:]
  get_storename = re.findall(r'target="_blank">\n(.+?)\n</a>',r_new.text)
  get_storename = rename(get_storename)
  get_monthdealnum = re.findall(r'>([0-9]+?笔)</em>',r_new.text)
  get_commentnum = re.findall(r'">([0-9]+?|[0-9]+?.[0-9]+?万)</a></span>',r_new.text)
  tem = np.vstack((get_itemname,get_price,get_storename,get_monthdealnum,get_commentnum))
  return tem

In [ ]:
# This block is so messy because I wanted to get all pages at first, but failed unluckily
# When dealing with one single page, things are quite simple, you just need to stack the array together
def pick_data_re(r_new):
  temp = np.array(['item_name','price','store_name','deal_number_in_last_month','comment_number'])
  # get_pagenum = re.findall(r'<b class="ui-page-s-len">[0-9]+?/([0-9]+?)</b>',r_new.text)
  # total_pagenum = eval(get_pagenum[0])
  # for i in range(1,total_pagenum):
  tem = get_one_page(r_new)
  temp = np.vstack((temp,tem.T))
    # break
    # if i == total_pagenum - 1: # when last page
    #   break 
    # if len(tem) == 0:
    #   print('get nothing!')
    #   break
    # url_new = re.findall(r'<a href="(.+?)" class="ui-page-s-next"',r_new.text)
    # url_new = re.findall(r'<a href="(.+?)" class="ui-page-s-next"',r.text)
    # r_new = requests.get(url+url_new[0].replace('amp;',''),headers={'User-Agent':'baiduspider'})
    # try:
    #   url_new = 'https://list.tmall.com/search_product.htm?spm=a220m.1000858.1000724.10.157a749foPy1g4&cat=50024411&s='+str(60*i)+'&q='+keyword+'&sort=s&style=g&industryCatId=50024411&type=pc#J_Filter'
    #   r_new = requests.get(url_new,headers=headers)
    #   r.encoding = 'gbk'
    #   # print('The %d time to request succeed.'%i)
    #   print(r_new.request.url)
    # except:
    #   print('The %d time to request failed.'%i)
  # print(len(get_price))
  # print(len(get_itemname))
  # print(len(get_storename))
  # print(len(get_monthdealnum))
  # print(len(get_commentnum))
  return temp

In [ ]:
# Main
r = getHtmlText_re(url)
result = pick_data_re(r)
np.savetxt('Price_Compare_of_TaoBao_Commodity.csv',result,fmt='%s',delimiter=',')

The First time to request succeed.


##**Further method to crawl News Web**

In [ ]:
# install pakeage selenium
pip install selenium

In [ ]:
# DownLoad chromedriver.exe from web
import os, requests
import zipfile

url = "https://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_win32.zip"
fname = "chromedriver_win32.zip"
if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)
extracting = zipfile.ZipFile('chromedriver_win32.zip')
extracting.extractall()

In [ ]:
# use selenium to crawl web
# basic version

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# the Web we want to crawl
mainurl = 'http://rettc.ustb.edu.cn'

# Virtual browser Options setting
chrome_options = Options()
chrome_options.add_argument('--headless')
prefs = {"profile.managed_default_content_settings.images":2}
chrome_options.add_experimental_option("prefs",prefs)

# For unknown reason, the line below can't run correctly
# But it can work using local verison, which changes the root to 'D:/***/chromedriver.exe'
############################################################################
driver = webdriver.Chrome("./chromedriver.exe",options = chrome_options)
############################################################################

# Data saving
f = open('./result.csv','a')
f.write('News_name'+','+'News_link'+'\n')
def save(n,h):
  f.write(n+','+h+'\n')

# use DOM node to get web content
def obtain_data():
  driver.get(mainurl)
  driver.set_page_load_timeout(10)
  nameList = driver.find_elements_by_css_selector('.news-wrap > .news-list > li > a')
  for i in range(0,len(nameList)):
    name = driver.find_elements_by_css_selector('.news-wrap > .news-list > li > a')[i].text
    href = driver.find_elements_by_css_selector('.news-wrap > .news-list > li > a')[i].get_attribute('href')
    print(name,href)
    save(name,href)